<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       SeasonalNormalize and Unnormalize function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> SeasonalNormalize() takes a non-stationary series and normalizes the series by first dividing the series into cycles and intervals, then averaging and normalizing with respect to each interval over all cycles. This form of normalization is effective relative to eliminating non-stationary properties such as unit roots and periodicities.</p>
 
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following procedure is an example of how to use SeasonalNormalize():</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Detect the unit roots using DickeyFuller().</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use SeasonalNormalize() to create a series with potentially the unit roots eliminated.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use DickeyFuller() to verify that unit roots were eliminated from the newly-formed normalized series.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use ArimaEstimate() and ArimaValidate() to create an ARIMA model from the normalized series.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use ArimaForecast() to forecast the normalized series.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use Unnormalize() passing in the forecasted series and the original unnormalized series to produce a forecasted series with the effects of SeasonalNormalize() removed.</li>
 </p>
 
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we will just see the usage of SeasonalNormalize and Unnormalize functions</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDSeries,
    SeasonalNormalize,
    Unnormalize,
    Figure,
    plot,
    db_drop_view,
    remove_context,
    copy_to_sql,
    db_drop_table
    )

from teradatasqlalchemy.types import *

from teradataml import to_numeric
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_SeasonalNormalize_Unnnormalize_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_SalesForecastingUAF_cloud');"        # Takes 1 minute
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_SalesForecastingUAF_local');"        # Takes 2 minutes

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
sales_data = DataFrame(in_schema('DEMO_SalesForecastingUAF', 'Weekly_Sales'))
feature_data = DataFrame(in_schema('DEMO_SalesForecastingUAF', 'Features')).drop(['IsHoliday'], axis=1)
store_data = DataFrame(in_schema('DEMO_SalesForecastingUAF', 'Stores'))

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will join datasets to create the Analytic Data Set using tdml for basic dataframe manipulations</p>

In [ ]:
# Join store_data with sales_data
sales_data = (
    sales_data.join(store_data, on='Store', how='left', lprefix='t1', rprefix='t2')
    .drop(['t2_Store'], axis=1)
)
sales_data = sales_data.assign(Store=sales_data['t1_Store'])
sales_data = sales_data.drop(['t1_Store'], axis=1)

# Join feature_data with sales_data
sales_data = (
    sales_data.join(feature_data, on=['Store', 'Date'], how='left', lprefix='t1', rprefix='t3')
    .drop(['t3_Store'], axis=1)
    .drop(['t3_Date'], axis=1)
)
sales_data = sales_data.assign(Store=sales_data['t1_Store'])
sales_data = sales_data.assign(Date=sales_data['t1_Date'])
sales_data = sales_data.drop(['t1_Store'], axis=1)
sales_data = sales_data.drop(['t1_Date'], axis=1)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will creating time series identifier for partitioning the data. We will join the department and store ID which provides a unique ID for every time series to create a column for partitioning.</p>

In [ ]:
# Assign a new column 'idcols' based on string concatenation
sales_data = sales_data.assign(idcols=sales_data.Dept.str.strip() + '-' + sales_data.Store.str.strip())
sales_data = sales_data.assign(idcols=sales_data.idcols.cast(type_=VARCHAR(10)))

# Check the shape of the DataFrame
sales_data.shape

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The dataset we created contains more than 421k rows and 19 columns. This final dataset will be copied to Vantage database</p>

In [ ]:
copy_to_sql(df = sales_data, table_name = "az_sf_joined", if_exists = "replace")

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. SeasonalNormalize</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>SeasonalNormalize() takes a non-stationary series and normalizes the series by first dividing the series into cycles and intervals, then averaging and normalizing with respect to each interval over all cycles.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The first step is to convert the time series data into TDSeries, which is required for the input time series which are passed to the convolve function.</p>


In [ ]:
from teradataml import TDSeries, Resample

data_series_df = TDSeries(
    data=sales_data,
    id="idcols",
    row_index=("times"),
    row_index_style="TIMECODE",
    payload_field="Weekly_Sales",
    payload_content="REAL"
)
uaf_out1 = Resample(data=data_series_df,
                    interpolate='LINEAR',
                    timecode_start_value="TIMESTAMP '2010-02-05 00:00:00'",
                    timecode_duration="WEEKS(1)")

df=uaf_out1.result
df1=df.select(['idcols','ROW_I', 'Weekly_Sales']).assign(Sales_Date=df.ROW_I)
df1

In [ ]:
data_series_df_norm = TDSeries(data=df1,
                              id="idcols",
                              row_index="Sales_Date",
                              row_index_style="TIMECODE",
                              payload_field="Weekly_Sales",
                              payload_content="REAL",
                              interval="WEEKS(1)") 

In [ ]:
# Normalize the series by removing the unit roots.
from teradataml import SeasonalNormalize
uaf_out = SeasonalNormalize(data=data_series_df_norm,
                                season_cycle="WEEKS",
                                cycle_duration=1,
                                output_fmt_index_style = 'FLOW_THROUGH')
norm_df = uaf_out.result
norm_df

In [ ]:
uaf_out.metadata

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Unnormalize</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Unnormalize() function reconstructs a series created by SeasonalNormalize(). The function is usually used for the forecasting phase of modeling.</p>
 
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following procedure is an example of how to use function when the series to be modeled is found to be unstationary:</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use SeasonalNormalize() to make the series stationary.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Develop the ARIMA forecast model.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use the ARIMA model to forecast the normalized series.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use function on the forecasted normalized series to undo the effects of normalization and produce the final forecasted series result.</li>
          
</p>          

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The first step is to convert the time series data into TDSeries, which is required for the input time series which are passed to the convolve function.</p>


In [ ]:
# Create teradataml TDSeries objects.
td_series1 = TDSeries(data=uaf_out.result,
                          id="idcols",
                          row_index="ROW_I",
                          row_index_style="TIMECODE",
                          payload_field="Weekly_Sales",
                          payload_content="REAL",
                          interval="WEEKS(1)"
                          )

In [ ]:
# Example 1 : Function reverse the effects of normalization and
#             produce the final forecasted series result using TDSeries.
 
# Create teradataml TDSeries objects.
td_series2 = TDSeries(data=uaf_out.metadata,
                          id="idcols",
                          row_index="ROW_I",
                          row_index_style="SEQUENCE",
                          payload_field=["MEAN_Weekly_Sales", "SD_Weekly_Sales"],
                          payload_content="MULTIVAR_REAL"
                          )

In [ ]:
uaf_out = Unnormalize(data1=td_series1,
                          data2=td_series2,
                          input_fmt_input_mode="MATCH",
                          output_fmt_index_style="FLOW_THROUGH")
 
# Print the result DataFrame.
unnorm_df=uaf_out.result
unnorm_df

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up intermediate tables.</p>

In [ ]:
db_drop_table(table_name='az_sf_joined')

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_SalesForecastingUAF');" 
#Takes 45 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>SeasonalNormalize function reference: <a href = 'https://docs.teradata.com/search/all?query=SeasonalNormalize&content-lang=en-US'>here</a></li>
    <li>Unnormalize function reference: <a href = 'https://docs.teradata.com/search/all?query=Unnormalize&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>